# ChArUco

- ref: https://github.com/kyle-bersani/opencv-examples
- don't use digital smoothing!

- `ffmpeg -i _back.mp4 -vf "crop=w=1440:h=1920:x=1440:y=0, transpose=2" back.mp4`
- `ffmpeg -i main.mp4 -map 0 -c copy -f segment -segment_time 60 -reset_timestamps 1 main_%03d.mp4`
- `python get_intrinsics.py --vid ../data/record3d/tonpy-v8/raw/back.mp4 --squares 10 7`

In [9]:
import cv2
import cv2.aruco as aruco
import imageio
import numpy as np
import matplotlib.pyplot as plt
import imageio
from copy import deepcopy
from tqdm import tqdm
import pickle

from utils import undistort


aruco_dict = aruco.Dictionary_get(aruco.DICT_4X4_1000)
charuco = aruco.CharucoBoard_create(
        squaresX=10,
        squaresY=7,
        squareLength=0.039,
        markerLength=0.0195,
        dictionary=aruco_dict)

In [10]:
with open('camera_raw.pkl', 'rb') as f:
    (mtx, dist) = pickle.load(f)

vid = imageio.mimread("../data/record3d/tonpy-v8/raw/back.mp4", memtest=False)
vid = undistort(vid, mtx, dist)

with open('camera_undist.pkl', 'rb') as f:
    (mtx, dist) = pickle.load(f)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 671/671 [00:14<00:00, 47.83it/s]


In [11]:
vid_sample, vid_paint, rvec_hist, tvec_hist = [], [], [], []

for t, img in enumerate(vid[30:-30:3]):
    bak = deepcopy(img)
    img = deepcopy(img[...,::-1])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    aruco_params = aruco.DetectorParameters_create()
    corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, aruco_dict, parameters=aruco_params)

    corners, ids, rejectedImgPoints, recoveredIds = aruco.refineDetectedMarkers(
            image = gray,
            board = charuco,
            detectedCorners = corners,
            detectedIds = ids,
            rejectedCorners = rejectedImgPoints,
            cameraMatrix = mtx,
            distCoeffs = dist)

    img = aruco.drawDetectedMarkers(img, corners, borderColor=(0, 0, 255), ids=ids)

    if not (ids is not None and len(ids) > 1):
        print("skip (t={}, ids={})".format(t, ids))
        continue

    response, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(
            markerCorners=corners,
            markerIds=ids,
            image=gray,
            board=charuco)

    if response < 8:
        print("skip (t={}, response={})".format(t, response))
        continue

    # valid, rvec, tvec = aruco.estimatePoseCharucoBoard(
    #         charucoCorners=charuco_corners,
    #         charucoIds=charuco_ids,
    #         board=charuco, 
    #         cameraMatrix=mtx,
    #         distCoeffs=dist,
    #         rvec=None,
    #         tvec=None)
    
    objp = np.empty((0,3), np.float32)
    for idx in charuco_ids:
        objpi = np.dot(charuco.chessboardCorners[idx] - np.array([[5,7,0]]) * 0.04, np.array([[-1,0,0], [0,1,0], [0,0,-1]]))
        objp = np.append(objp, objpi, axis=0)

    valid, rvec, tvec = cv2.solvePnP(objp, charuco_corners, mtx, dist)

    if not valid:
        print("skip (t={}, valid={})".format(t, valid))
        continue

    img = cv2.drawFrameAxes(img, mtx, dist, rvec, tvec, 0.3)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    vid_sample.append(bak)
    vid_paint.append(img)
    rvec_hist.append(rvec)
    tvec_hist.append(tvec)

print(len(vid_sample))
imageio.mimwrite("temp.mp4", vid_paint, macro_block_size=8)

skip (t=0, ids=[[3]])
skip (t=1, response=0)
skip (t=2, response=0)
skip (t=3, response=0)
skip (t=4, response=0)
skip (t=5, ids=[[17]])
skip (t=6, ids=None)
skip (t=7, response=0)
skip (t=8, ids=[[8]])
skip (t=9, ids=[[17]])
skip (t=10, response=0)
skip (t=11, response=0)
skip (t=12, response=0)
skip (t=13, response=1)
skip (t=14, response=2)
skip (t=15, ids=[[17]])
skip (t=16, response=1)
skip (t=17, ids=None)
skip (t=18, response=0)
skip (t=19, ids=[[17]])
skip (t=20, ids=None)
skip (t=21, response=0)
skip (t=22, ids=None)
skip (t=23, ids=[[22]])
skip (t=24, ids=None)
skip (t=25, response=1)
skip (t=26, ids=None)
skip (t=27, ids=[[37]])
skip (t=28, response=0)
skip (t=29, response=0)
skip (t=30, ids=None)
skip (t=31, response=0)
skip (t=32, response=0)
skip (t=33, ids=[[16]])
skip (t=34, ids=[[22]])
skip (t=35, response=0)
skip (t=36, response=0)
skip (t=37, ids=[[22]])
skip (t=38, ids=[[517]])
skip (t=39, response=0)
skip (t=40, ids=[[930]])
skip (t=41, ids=[[22]])
skip (t=42, ids=

In [13]:
imageio.mimwrite("src/back_real.mp4", vid_sample, macro_block_size=8)

In [4]:
!rm -rf images

In [6]:
vid_paint[0].shape

(1440, 1920, 3)

In [7]:
w = 1920
h = 1440  # 1080

import os
import PIL
import json
import torch

os.makedirs('images/', exist_ok=True)

out = {
    "fl_x": float(mtx[0,0]),
    "fl_y": float(mtx[1,1]),
    "cx": float(mtx[0,2]),
    "cy": float(mtx[1,2]),
    "w": w,
    "h": h,
    "camera_model": 'OPENCV',
    "k1": dist[0,0],
    "k2": dist[0,1],
    "p1": dist[0,2],
    "p2": dist[0,3],
}

poses = []
for t, (rvec, tvec) in enumerate(zip(rvec_hist, tvec_hist)):
    R, _ = cv2.Rodrigues(rvec)
    T = tvec

    c2w = np.eye(4)
    c2w[:3,3] = np.dot(R.T, - T).squeeze()
    c2w[:3,:3] = np.dot(R.T, np.array([[1,0,0], [0,-1,0], [0,0,-1]]))
    poses.append(c2w)


poses = torch.from_numpy(np.array(poses).astype(np.float32))
print(torch.max(torch.abs(poses[:, :3, 3])))


tensor(0.8409)


In [8]:
# scale_factor = 1.0
# scale_factor /= torch.max(torch.abs(poses[:, :3, 3]))
# print(scale_factor)
# poses[:, :3, 3] *= scale_factor * self.config.scale_factor

In [ ]:
frames = []

for t, (img, c2w) in enumerate(zip(vid_sample, poses)):
    name = 'images/frame_{:05}.png'.format(t)
    PIL.Image.fromarray(img).save(name, quality=95)  # heavy

    frame = {
        "file_path": name,
        "transform_matrix": c2w.tolist(),
    }
    frames.append(frame)

out["frames"] = frames

with open("transforms.json", "w", encoding="utf-8") as f:
    json.dump(out, f, indent=4)

In [14]:
len(frames)

96

```
dataname/
  ├ raw/
  │   ├ back.mp4
  │   └ main.mp4
  └ colmap/
      ├ images/
      └ transforms.json
```